#### Библиотеки

In [1]:
import sys

sys.path.append("..")

import warnings

import pandas as pd
import torch
from torch.nn import functional as F
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoModelForCausalLM
)

import os

from warp.utils.misc import seed_everything
from warp.utils.data import prepare_warp_dataset
from warp.constants import DATASET_DIR, CONFIG_DIR, MODEL_DIR
from pathlib import Path

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

На этот раз нам нужны две модели. Одну из них мы предположительно обучили в прошлом ноутбуке. Если по какой-то причине она не лежит в `artifacts/reward_model`, можно скачать какую-то другую

In [45]:
model_name = Path(MODEL_DIR, "reward_model")
# model_name = "lvwerra/distilbert-imdb"

reward_tokenizer = AutoTokenizer.from_pretrained(
    model_name, max_length=512, use_fast=True
)
reward_model = AutoModelForSequenceClassification.from_pretrained(
    model_name
).to(device)

Проверим, как она работает. Помним, что наша главная цель - доставать reward

In [48]:
reward_model(
    **reward_tokenizer(
        [
            "this movie is brilliant",
            "this movie is so bad",
            "this movie was so so",
        ],
        return_tensors="pt",
        padding=True,
        truncation=True
    )
).logits

tensor([[-4.8782, -4.9536],
        [ 4.7472,  4.5910],
        [ 0.9542,  0.8615]], grad_fn=<AddmmBackward0>)

Как я и писал (надеюсь) ранее, сами по себе логиты нам вообще никак не помогут, даже если дать сверху софтмакс. Reward нужно считать друг относительно друга, для этого нужна отдельная функция, по аналогии с той, что есть в `RewardTrainer`

In [52]:
def get_reward(prompt, response):
    return (
        torch.stack(
            [
                reward_model(
                    **reward_tokenizer(
                        input,
                        padding=True,
                        return_tensors="pt",
                    )
                ).logits.detach()
                for input in [prompt, response]
            ]
        )
        .mean(dim=2)
        .softmax(0)
    )

In [53]:
get_reward(
    ["this movie is"],
    ["this movie is so damn cool, bro. i've never seen anything better in my life"],
)

tensor([[0.9821],
        [0.0179]])

Как видим, всё отлично - положительные отзывы мы не хотим. Вопрос только в том, что $r$ в статье отрицательный, а $r_{\beta}$ как будто бы может принимать любые значения, так что софтмакс здесь тоже находится под вопросом. Итого, есть 4 варианта, как понять статью, я пробовал все

В любом случае нам нужна SFT-модель

In [21]:
model_name = Path(MODEL_DIR, "lvwerra/gpt2-imdb")

sft_tokenizer = AutoTokenizer.from_pretrained(
    model_name, max_length=512, use_fast=True
)
sft_tokenizer.pad_token = sft_tokenizer.eos_token
sft_model = AutoModelForCausalLM.from_pretrained(
    model_name, return_dict_in_generate=True
).to(device)

В статье указывается только про температуру, её мы пропишем. Уточняется, что они дообучали 28 слоёв, но в самой модели их чуть больше. Хотя они вроде не уточняют, что они брали

In [3]:
i = 0
for param in sft_model.parameters():
    if param.requires_grad:
        i += 1
i

148

In [22]:
from peft import get_peft_model, TaskType, LoraConfig

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False, 
    r=16, 
    lora_alpha=32, 
    lora_dropout=0.05,
    fan_in_fan_out=True
)
get_peft_model(sft_model, lora_config)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lo

In [24]:
i = 0
for param in sft_model.parameters():
    if param.requires_grad:
        i += 1
i

24

Гораздо лучше, это и попробуем тюнить. Сразу скажу, что без адаптера я тоже пробовал, но модель довольно сильно плывём. Хочется думать, что основные её веса норм

Следующий шаг - научиться находить полиси $\pi(y|x)$. Насколько я понимаю, достаточно найти вероятность встретить каждый токен при наличии контекста, а затем перемножить. Это вообще можно достать из генерации, но прикол в том, что полиси нужно оценивать у двух разных моделей, для этого нужно похитрить

In [12]:
def get_policy(model, input_ids, len_generated):

    logits = model(input_ids).logits
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = input_ids[..., 1:].contiguous()
    policy = -F.cross_entropy(
        shift_logits.transpose(1, 2), shift_labels, reduction="none"
    )[:, -len_generated:].sum(-1)

    return policy

96

У нас в сетапе есть $\theta$ и $\theta_{\text{ema}}$. У них один и тот же токенизатор, что гарантирует, что инпуты они обработают одинаково, это хорошо. Логиты можно достать, если прогнать форвард. Тогда останется посчитать кросс-энтропию от того, что мы нагенерили, получить то, что есть, то есть буквально $(x_n|x_1, ..., x_{n-1})$. Чтобы получить вероятность именно всего предложения, просуммируем логвероятности, как в обычной языковой модели. `len_generated` нужен, чтобы считать вероятность именно генерации. Вообще это костыль и его надо сделать красивее, потому что есть паддинг. Но в нашем сетапе длина везде 10, так что я забил

In [ ]:
input_ids = sft_tokenizer(["this movie is so"], return_tensors="pt")
response_input_ids = sft_model.generate(input_ids, output_scores=True)
get_policy(response_input_ids, input_ids, 5)

Следующий шаг - научится делать интерполяцию. Начнём вот с чего:

1. Методы интерполяции в статье делаются для векторов. Веса в сетках, к соэалению, обычно матрицы. Как этого избежать? Я не знаю, но я считал, что каждая матрица это кортеж векторов, каждый из которых я интерполирую
2. SLERP делается для двух сущностей, а нам надо интерполировать много. В аппендиксе, если я правильно понял, они итеративно применяют его к каждой паре. Только применять его можно по-разному, например:
$$
\theta_{\text{init}} = \theta_{\text{init}} + \lambda_1\theta_{1} + \lambda_2\theta_{2} \\
\theta_{\text{init}} = \theta_{\text{init}} + \lambda_1(\theta_{\text{init}} + \lambda_2\theta_{2}) + ...\theta_{3}
$$
То есть `slerp(slerp(theta, 1, 2), 3)`. Я же выбрал другой вариант, который, как по мне разумнее. Будем идти окном и мёрджить веса попарно. Изначальный вес добавим в самом конце
$$
\theta_{2} = \lambda_1\theta_{1} + \lambda_2\theta_{2} \\
... \\
\theta_{m} = \lambda_1\theta_{m-1} + \lambda_2\theta_{m} \\
\theta_{\text{init}} = \theta_{\text{init}} + \theta_{m}
$$
В общем, тут можно поразмышлять, как сделать лучше. Мне кажется, что это ни на что особо не повлияет, проверить к сожалению не успею, но вот идейка, как можно модернизировать

3. Ручные обновления параметров надо сделать чуть покрасивее, чем это сделал сейчас я, тут каюсь, есть куда расти, но они лежат в тренере, тут я их показывать не буду

Так что остаётся лишь взглянуть на SLERP и поедем тюнить модельку

In [26]:
def slerp(theta, thetas, lamb):

    for i in range(len(thetas) - 1):

        delta_1 = thetas[i] - theta
        delta_2 = thetas[i + 1] - theta

        omega = (
            torch.einsum(
                "ij, ij -> i",
                delta_1 / delta_1.norm(p=2, dim=1, keepdim=True),
                delta_2 / delta_2.norm(p=2, dim=1, keepdim=True),
            )
            .unsqueeze(-1)
            .arccos()
        )

        thetas[i + 1] = (
            (torch.sin((1 - lamb) * omega) / torch.sin(omega)) * delta_1
            + (torch.sin(lamb * omega) / torch.sin(omega)) * delta_2
        )

    return theta + thetas[-1]

Прикольное упражнение на подумать - почему эта имплементация не очень, ответы в скрипте `warp/utils/train.py`

In [35]:
theta = torch.rand(3, 2)
theta_1 = torch.rand(3, 2)
theta_2 = torch.rand(3, 2)

In [36]:
slerped = slerp(theta, [theta_1, theta_2], lamb=0.5)
assert slerped.shape == theta.shape

In [37]:
slerped, theta, theta_1, theta_2

(tensor([[0.8703, 0.5297],
         [0.1625, 0.7183],
         [0.9897, 0.5562]]),
 tensor([[0.5833, 0.3161],
         [0.8885, 0.7056],
         [0.5693, 0.4831]]),
 tensor([[0.4092, 0.5345],
         [0.4266, 0.4700],
         [0.8235, 0.8636]]),
 tensor([[9.5895e-01, 2.4779e-01],
         [8.0824e-04, 9.6473e-01],
         [8.3183e-01, 1.9243e-01]]))

Ну, об адекватности судить сложно, но по крайней мере оно работает

Осталось только собрать датасет

In [38]:
train, test = load_dataset("imdb", split=["train", "test"])
train, test = [pd.DataFrame(dataset) for dataset in [train, test]]
train

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


Как и сказано в задании, возьмём первые 10 токено, только и всего. Важно, что токенизировать их будем через SFT, потому что их же будем пихать в SFT-модель

In [43]:
import polars as pl

what = prepare_warp_dataset(
    pl.DataFrame(train).to_dict(as_series=False),
    tokenizer=sft_tokenizer,
    max_length=10,
)
what

2024-08-05 17:48:23.956 | INFO     | warp.utils.data:prepare_warp_dataset:164 - Starting tokenizing `text`


Dataset({
    features: ['input_ids', 'attention_mask', 'text'],
    num_rows: 25000
})

In [42]:
what[0]

{'input_ids': tensor([   40, 26399,   314,  3001,   327, 47269, 20958,    12,    56, 23304]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><

In [21]:
sft_tokenizer.batch_decode(
    sft_model.generate(
        temperature=0.9,
        **sft_tokenizer(["this movie is"], return_tensors="pt"),
    ).sequences
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['this movie is a waste of time. I was very disappointed. I was very disappointed. I was']

Наконец, остаётся собрать трейн луп. Я бы с удовольствием сделал его как-нибудь красиво через lightning, но тут довольно много всего, так что внес в отдельный класс, который можно (и нужно) посмотреть в `warp/structs/warp_trainer.py`. Если я нигде не ошибся в реализации, то и сам алгоритм чисто теоретически должен работать. Запустить весь процесс можно командой ниже. Как обычно, рекомендую предварительно ознакомиться с параметрами в `warp/configs/warp_config.yaml`

Основные параметры - как просят в задании. Оставшиеся вопросы - lr оптимайзера, потому что он очень важен, и scheduler. Они говорят, что есть warmup, но на этом всё. Я импортнул подобный из лайтнинга

In [ ]:
!poetry run python warp/train_warp.py --config-name warp_config